In [ ]:
import time
import requests
import re
from pprint import pformat
from json import dumps,loads
from kafka import KafkaConsumer, KafkaProducer

In [ ]:
TOPIC_Step1_NAME="Sahamyab-Session_16"
TOPIC_Step2_NAME="Sahamyab-Session_16_2"
KAFKA_SERVER="kafka-broker:29092"

In [ ]:
def get_tags(text):
    tags = re.findall(r"#(\w+)", text)
    #  r"#(\w+)" matches all occurrences of a hashtag followed by one or more word characters
    return tags

In [ ]:
consumer = KafkaConsumer(
     TOPIC_Step1_NAME, #This is the name of the Kafka topic from which the consumer will read data.
     bootstrap_servers=[KAFKA_SERVER],
     auto_offset_reset='earliest',
     enable_auto_commit=True,
     group_id='hashtag-group',
     value_deserializer=lambda x: loads(x.decode('utf-8')),
     key_deserializer=lambda x : str(x),
     client_id='consumer-of-tweets'
)

In [ ]:
#  key_deserializer=lambda x : str(x), This function is called to deserialize the key of each message. Here, it’s converting each key to a string

In [ ]:
producer = KafkaProducer(bootstrap_servers=['kafka-broker:29092'], value_serializer=lambda x: dumps(x).encode('utf-8'),
                        key_serializer=str.encode)

In [ ]:
# This cell processes messages from a Kafka consumer
cnt=1
for message in consumer:
    try : 
        data=message.value
        del data['raw-data']
        print("-=-"*20)
        print(f"#{cnt:4} - Processing Tweet ID {message.key}")
        tags=get_tags(data['content'])
        print(f"Tags : {tags}")
        data["hashtags"] = tags
        producer.send(TOPIC_Step2_NAME, value=data, key= data['id'])
        cnt+=1
    except Exception as ex:
        print("%%%-"*20)
        print(ex)
        
"""        
This code reads messages from a Kafka consumer, extracts hashtags from the message content,
updates the message data with the extracted hashtags, and sends the updated data to a Kafka
producer for further processing!        
"""